In [66]:
import shutil
import os, sys, random
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
from shutil import copyfile
import pandas as pd
from sklearn import preprocessing, model_selection
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import patches
import numpy as np
import os
from sklearn.model_selection import train_test_split


In [67]:
annotations = sorted(glob('/Users/iliyask/Helmet_iliyas/helmet_det/VOC2028/Annotations/*.xml'))

In [68]:
df = []
cnt = 0
for file in annotations:
  prev_filename = file.split('/')[-1].split('.')[0] + '.jpg'
  filename = str(cnt) + '.jpg'
  row = []
  parsedXML = ET.parse(file)
  for node in parsedXML.getroot().iter('object'):
    helmet = node.find('name').text
    xmin = int(node.find('bndbox/xmin').text)
    xmax = int(node.find('bndbox/xmax').text)
    ymin = int(node.find('bndbox/ymin').text)
    ymax = int(node.find('bndbox/ymax').text)

    row = [prev_filename, filename, helmet, xmin, xmax, ymin, ymax]
    df.append(row)
  cnt += 1


In [69]:
data = pd.DataFrame(df, columns=['prev_filename', 'filename', 'helmet_person', 'xmin', 'xmax', 'ymin', 'ymax'])

data=data[data['helmet_person']!='dog']

In [70]:
data['helmet_person'].value_counts()

helmet_person
person    111514
hat         9044
Name: count, dtype: int64

In [71]:
data

,prev_filename,filename,helmet_person,xmin,xmax,ymin,ymax
0,000000.jpg,0.jpg,hat,60,910,66,1108
1,000002.jpg,1.jpg,hat,37,76,32,84
2,000002.jpg,1.jpg,hat,97,125,44,82
3,000002.jpg,1.jpg,hat,165,208,103,158
4,000002.jpg,1.jpg,hat,178,213,71,113
...,...,...,...,...,...,...,...
120556,part2_002394.jpg,7580.jpg,hat,14,117,22,126
120557,part2_002394.jpg,7580.jpg,hat,312,345,81,121
120558,part2_002394.jpg,7580.jpg,hat,366,401,82,130
120559,part2_002394.jpg,7580.jpg,hat,415,525,47,153


In [72]:
data[['prev_filename','filename', 'helmet_person', 'xmin', 'xmax', 'ymin', 'ymax']].to_csv('/Users/iliyask/Helmet_iliyas/helmet_det/output.csv', index=False)


In [73]:
# TODO read actual image size
img_width = 640
img_height = 480

def width(df):
  return int(df.xmax - df.xmin)
def height(df):
  return int(df.ymax - df.ymin)
def x_center(df):
  return int(df.xmin + (df.width/2))
def y_center(df):
  return int(df.ymin + (df.height/2))
def w_norm(df):
  return df/img_width
def h_norm(df):
  return df/img_height

df = pd.read_csv('/Users/iliyask/Helmet_iliyas/helmet_det/output.csv')

le = preprocessing.LabelEncoder()
le.fit(df['helmet_person'])
print(le.classes_)
labels = le.transform(df['helmet_person'])
df['labels'] = labels

df['width'] = df.apply(width, axis=1)
df['height'] = df.apply(height, axis=1)

df['x_center'] = df.apply(x_center, axis=1)
df['y_center'] = df.apply(y_center, axis=1)

df['x_center_norm'] = df['x_center'].apply(w_norm)
df['width_norm'] = df['width'].apply(w_norm)

df['y_center_norm'] = df['y_center'].apply(h_norm)
df['height_norm'] = df['height'].apply(h_norm)

df.head(30)


['hat' 'person']


,prev_filename,filename,helmet_person,xmin,xmax,ymin,ymax,labels,width,height,x_center,y_center,x_center_norm,width_norm,y_center_norm,height_norm
0,000000.jpg,0.jpg,hat,60,910,66,1108,0,850,1042,485,587,0.757812,1.328125,1.222917,2.170833
1,000002.jpg,1.jpg,hat,37,76,32,84,0,39,52,56,58,0.087500,0.060937,0.120833,0.108333
2,000002.jpg,1.jpg,hat,97,125,44,82,0,28,38,111,63,0.173437,0.043750,0.131250,0.079167
3,000002.jpg,1.jpg,hat,165,208,103,158,0,43,55,186,130,0.290625,0.067187,0.270833,0.114583
4,000002.jpg,1.jpg,hat,178,213,71,113,0,35,42,195,92,0.304688,0.054688,0.191667,0.087500
5,000002.jpg,1.jpg,hat,221,251,44,88,0,30,44,236,66,0.368750,0.046875,0.137500,0.091667
6,000002.jpg,1.jpg,hat,249,283,61,112,0,34,51,266,86,0.415625,0.053125,0.179167,0.106250
7,000002.jpg,1.jpg,hat,335,376,60,112,0,41,52,355,86,0.554688,0.064062,0.179167,0.108333
8,000002.jpg,1.jpg,hat,344,385,107,163,0,41,56,364,135,0.568750,0.064062,0.281250,0.116667
9,000002.jpg,1.jpg,hat,372,402,59,110,0,30,51,387,84,0.604688,0.046875,0.175000,0.106250


In [74]:
len(set(df['prev_filename']))

7581

In [75]:
len(set(df['filename']))

7581

In [76]:
unique_values = df['prev_filename'].unique()

# 2. Split these unique values into training and testing sets
df_train1, df_test = train_test_split(unique_values, test_size=0.1, random_state=13, shuffle=True)
# 3. Create the training and testing DataFrames based on these splits
df_train1 = df[df['prev_filename'].isin(df_train1)]
df_test = df[df['prev_filename'].isin(df_test)]


unique_values_train = df_train1['prev_filename'].unique()

# 2. Split these unique values into training and testing sets
df_train, df_val = train_test_split(unique_values_train, test_size=0.1, random_state=13, shuffle=True)
# 3. Create the training and testing DataFrames based on these splits
df_train = df_train1[df_train1['prev_filename'].isin(df_train)]
df_val = df_train1[df_train1['prev_filename'].isin(df_val)]


In [77]:


os.mkdir('Data')

os.mkdir('Data/images/')
os.mkdir('Data/images/train/')
os.mkdir('Data/images/test/')
os.mkdir('Data/images/val/')

os.mkdir('Data/labels/')
os.mkdir('Data/labels/train/')
os.mkdir('Data/labels/test/')
os.mkdir('Data/labels/val')


In [78]:

def segregate_data(df, img_path, label_path, train_img_path, train_label_path):
  filenames = []
  for filename in df.filename:
    filenames.append(filename)
  filenames = set(filenames)

  for filename in filenames:
    yolo_list = []

    for _,row in df[df.filename == filename].iterrows():
      yolo_list.append([row.labels, row.x_center_norm, row.y_center_norm, row.width_norm, row.height_norm])

    yolo_list = np.array(yolo_list)
    txt_filename = os.path.join(train_label_path,str(row.prev_filename.split('.')[0])+".txt")
    # Save the .img & .txt files to the corresponding train and validation folders
    np.savetxt(txt_filename, yolo_list, fmt=["%d", "%f", "%f", "%f", "%f"])
    shutil.copyfile(os.path.join(img_path,row.prev_filename), os.path.join(train_img_path,row.prev_filename))

In [79]:
## Apply function ##

src_img_path = "/Users/iliyask/Helmet_iliyas/helmet_det/VOC2028/JPEGImages"
src_label_path = "/Users/iliyask/Helmet_iliyas/helmet_det/VOC2028/Annotations"

train_img_path = "/Users/iliyask/Helmet_iliyas/helmet_det/Data/images/train"
train_label_path = "/Users/iliyask/Helmet_iliyas/helmet_det/Data/labels/train"

test_img_path = "/Users/iliyask/Helmet_iliyas/helmet_det/Data/images/test"
test_label_path = "/Users/iliyask/Helmet_iliyas/helmet_det/Data/labels/test"

val_img_path = "/Users/iliyask/Helmet_iliyas/helmet_det/Data/images/val"
val_label_path = "/Users/iliyask/Helmet_iliyas/helmet_det/Data/labels/val"

segregate_data(df_train, src_img_path, src_label_path, train_img_path, train_label_path)
segregate_data(df_test, src_img_path, src_label_path, test_img_path, test_label_path)
segregate_data(df_val, src_img_path, src_label_path, val_img_path, val_label_path)


In [80]:
print("No. of Training images", len(os.listdir(train_img_path)))
print("No. of Training labels", len(os.listdir(train_label_path)))

print("No. of test images", len(os.listdir(test_img_path)))
print("No. of test labels", len(os.listdir(test_label_path)))

print("No. of valid images", len(os.listdir(val_img_path)))
print("No. of valid labels", len(os.listdir(val_label_path)))

No. of Training images 6139
No. of Training labels 6139
No. of test images 759
No. of test labels 759
No. of valid images 683
No. of valid labels 683


In [81]:
len(set(df['filename']))

7581

In [82]:
len(set(df_test['filename']))

759

In [83]:
len(set(df_train['filename']))

6139

In [84]:
len(set(df_val['filename']))

683

In [85]:
len(set(df_test['filename']))+ len(set(df_train['filename']))+len(set(df_val['filename']))

7581

In [89]:
df_train['helmet_person'].value_counts(normalize=True)

helmet_person
person    0.923008
hat       0.076992
Name: proportion, dtype: float64

In [87]:
df_test['helmet_person'].value_counts(normalize=True)

helmet_person
person    0.930728
hat       0.069272
Name: proportion, dtype: float64

In [90]:
df_val['helmet_person'].value_counts(normalize=True)

helmet_person
person    0.936216
hat       0.063784
Name: proportion, dtype: float64

In [93]:
pip install ultralytics


  Using cached ultralytics-8.2.18-py3-none-any.whl.metadata (40 kB)
  Using cached opencv_python-4.9.0.80-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
  Using cached torch-2.3.0-cp310-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached torchvision-0.18.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl.metadata (2.7 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached ultralytics-8.2.18-py3-none-any.whl (757 kB)
Using cached opencv_python-4.9.0.80-cp37-abi3-macosx_11_0_arm64.whl (35.4 MB)
Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
Using cached torch-2.3.0-cp310-none-macosx_11_0_arm64.whl (61.0 MB)
Using cached torchvision-0.18.0-cp310-cp310-macosx_11_0_arm64.whl (1.6 MB)
Using cac

In [99]:
from ultralytics import YOLO


In [101]:
# Load a model
model = YOLO("yolov5n.yaml")  # build a new model from scratch

# Use the model


In [108]:
results = model.train(data="helmet.yaml", epochs=1)  # train the model

Ultralytics YOLOv8.2.18 🚀 Python-3.10.6 torch-2.3.0 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov5n.yaml, data=helmet.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, form

train: Scanning /Users/iliyask/Helmet_iliyas/helmet_det/Data/labels/train... 6139 images, 0 backgrounds, 2926 corrupt: 100%|██████████| 6139/6139 [00:01<00:00, 3636.93it/s]

train: WARNING ⚠️ /Users/iliyask/Helmet_iliyas/helmet_det/Data/images/train/000000.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.2229      1.3281      2.1708]
train: WARNING ⚠️ /Users/iliyask/Helmet_iliyas/helmet_det/Data/images/train/000002.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/iliyask/Helmet_iliyas/helmet_det/Data/images/train/000003.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/iliyask/Helmet_iliyas/helmet_det/Data/images/train/000007.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/iliyask/Helmet_iliyas/helmet_det/Data/images/train/000011.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/iliyask/Helmet_iliyas/helmet_det/Data/images/train/000012.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0281]
train: WARNING ⚠️ /Users/iliyask/Helmet_iliyas/helmet_det/Data/images/train/000019.jpg: ignoring corrupt image/label: non-normalized or out of b


val: Scanning /Users/iliyask/Helmet_iliyas/helmet_det/Data/labels/val... 683 images, 0 backgrounds, 338 corrupt: 100%|██████████| 683/683 [00:00<00:00, 2436.61it/s]

val: WARNING ⚠️ /Users/iliyask/Helmet_iliyas/helmet_det/Data/images/val/000032.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0156        1.45      1.6109      1.8734]
val: WARNING ⚠️ /Users/iliyask/Helmet_iliyas/helmet_det/Data/images/val/000061.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/iliyask/Helmet_iliyas/helmet_det/Data/images/val/000115.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.7578      2.7344      1.0417      1.6656      1.2458      1.9333]
val: WARNING ⚠️ /Users/iliyask/Helmet_iliyas/helmet_det/Data/images/val/000173.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /Users/iliyask/Helmet_iliyas/helmet_det/Data/images/val/000186.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0094      1.1281      1.1844]
val: WARNING ⚠️ /Users/iliyask/Helmet_iliyas/helmet_det/Data/images/val/000230.jpg: ignoring corrupt image/label: non-normalized or 

Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 69 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train4
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      4.539      4.865      4.213        184        640:   6%|▋         | 13/201 [01:35<23:02,  7.36s/it]


KeyboardInterrupt: 